In [1]:
%matplotlib inline
%precision 7
import nfldb
import nflgame
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 2018 Draft Rankings by Position

In this notebook, we wish to use the nfldb to gather a set of features by positition that we think should be useful in prediciting player ranks. Once we decide the features, we will use the nfldb package to retrieve our data, averaging the data going back the past 3 years.

Before we begin, lets explore some of the nfldb syntax.

In [12]:
import nflgame

game = nflgame.one(2017, 1, "HOU", "JAX")
#for p in game.players:
#    print p.name, p.player

In [14]:
db = nfldb.connect()
q = nfldb.Query(db)

q.game(season_year=2017, season_type='Regular')
#for pp in q.sort('rushing_yds').limit(10).as_aggregate():
#    print pp.player, pp.rushing_yds

#for pp in q.as_games():
#    if pp.home_team == 'HOU' or pp.away_team == 'HOU':
#        print pp.week, pp.gsis_id

hou_game = nfldb.Game.from_id(db, '2017091005')
#for pp in hou_game.players:
#    print pp[1]

In [15]:
query = '''
SELECT
  *
FROM team
'''
players = []
with nfldb.Tx(db) as cursor:
    cursor.execute(query)
    for row in cursor.fetchall():
        print row

{'city': 'Arizona', 'team_id': 'ARI', 'name': 'Cardinals'}
{'city': 'Atlanta', 'team_id': 'ATL', 'name': 'Falcons'}
{'city': 'Baltimore', 'team_id': 'BAL', 'name': 'Ravens'}
{'city': 'Buffalo', 'team_id': 'BUF', 'name': 'Bills'}
{'city': 'Carolina', 'team_id': 'CAR', 'name': 'Panthers'}
{'city': 'Chicago', 'team_id': 'CHI', 'name': 'Bears'}
{'city': 'Cincinnati', 'team_id': 'CIN', 'name': 'Bengals'}
{'city': 'Cleveland', 'team_id': 'CLE', 'name': 'Browns'}
{'city': 'Dallas', 'team_id': 'DAL', 'name': 'Cowboys'}
{'city': 'Denver', 'team_id': 'DEN', 'name': 'Broncos'}
{'city': 'Detroit', 'team_id': 'DET', 'name': 'Lions'}
{'city': 'Green Bay', 'team_id': 'GB', 'name': 'Packers'}
{'city': 'Houston', 'team_id': 'HOU', 'name': 'Texans'}
{'city': 'Indianapolis', 'team_id': 'IND', 'name': 'Colts'}
{'city': 'Jacksonville', 'team_id': 'JAC', 'name': 'Jaguars'}
{'city': 'Kansas City', 'team_id': 'KC', 'name': 'Chiefs'}
{'city': 'Miami', 'team_id': 'MIA', 'name': 'Dolphins'}
{'city': 'Minnesota',

### Creating a Fantasy Scoring Function

Before creating our ranks, we need to implement a scoring function that translates stats into fantasy points. This scoring function will need to be modified depending on league settings.

In [16]:
scoring = {
    'passing_yds' : lambda x : x*.04,
    'passing_tds' : lambda x : x*4., 
    'passing_int' : lambda x : -2.*x,
    'rushing_yds' : lambda x : x*.1,
    'rushing_tds' : lambda x : x*6.,
    'kickret_tds' : lambda x : x*6.,
    'receiving_tds' : lambda x : x*6.,
    'receiving_yds' : lambda x : x*.1,
    'receiving_rec' : lambda x : .5*x,
    'fumbles_lost' : lambda x : -2.*x,
    'passing_twoptm'  : lambda x : 2.*x,
    'rushing_twoptm' : lambda x : 2.*x,
    'receiving_twoptm' : lambda x : 2.*x
}

def score_player(player_dict_agg):
    score = 0;
    for stat in scoring.keys():
        if player_dict_agg.has_key(stat) and player_dict_agg[stat] > 0:
            #print stat, player_dict_agg[stat], scoring[stat](player_dict_agg[stat])
            score += scoring[stat](player_dict_agg[stat])
    #print player_dict_agg['full_name'], player_dict_agg['position'], score, player_dict_agg['games_played']
    return round(score / player_dict_agg['games_played'], 1) 

### Ranking Quarterbacks Per Week

When ranking qbs, some things to consider would be avg_fp points per game, avg pass completion percentage, overall team offensive rank.

Lets gather the following to predict QB rankings - Looking back at last year and this year:

- Avg fp/game
- Avg Pass completion pct
- Avg team offense rank
- Avg team passing attempts rank

#### Avg FP/game

Lets write a function that gets the avg FP/game by position

In [45]:
db = nfldb.connect()
q = nfldb.Query(db)

def convert_list_sql_string(my_list):
    return ', '.join(map(lambda x: "'" + x + "'", my_list))

def get_qb_stats(year_list, pos_list):
    year_string = convert_list_sql_string(year_list)
    pos_string = convert_list_sql_string(pos_list)
    
    players = {}
    
    score_query = '''
    SELECT agg.*, player.full_name as full_name, player.position as position from
    (
        SELECT
          play_player.player_id,
          SUM(play_player.passing_yds) AS passing_yds,
          SUM(play_player.passing_tds) AS passing_tds,
          SUM(play_player.passing_int) AS passing_int,
          SUM(play_player.rushing_yds) AS rushing_yds,
          SUM(play_player.passing_att) as passing_att,
          SUM(play_player.passing_cmp) as passing_cmp,
          SUM(play_player.rushing_yds) AS rushing_yds,
          SUM(play_player.rushing_tds) AS rushing_tds,
          SUM(play_player.kickret_tds) AS kickret_tds,
          SUM(play_player.receiving_tds) AS receiving_tds,
          SUM(play_player.receiving_yds) AS receiving_yds,
          SUM(play_player.receiving_rec) AS receiving_rec,
          SUM(play_player.fumbles_lost) AS fumbles_lost,
          SUM(play_player.passing_twoptm) AS passing_twoptm,
          SUM(play_player.rushing_twoptm) AS rushing_twoptm,
          SUM(play_player.receiving_twoptm) AS receiving_twoptm,
          COUNT(DISTINCT game.gsis_id) AS games_played
        FROM play_player
        LEFT JOIN game ON play_player.gsis_id = game.gsis_id
        WHERE game.season_type = 'Regular' AND game.season_year in (%s)
        GROUP BY play_player.player_id
        HAVING COUNT(DISTINCT game.gsis_id) > 4
        AND COUNT(DISTINCT play_player.play_id) > 1
    ) as agg
    LEFT JOIN player on agg.player_id = player.player_id
    WHERE player.position in (%s)
    ''' % (year_string, pos_string)
    
    with nfldb.Tx(db) as cursor:
        cursor.execute(score_query)
        for row in cursor.fetchall():
            cmp_pct = 0
            if row['passing_att'] != 0:
                cmp_pct = round((row['passing_cmp']*1.0 / row['passing_att']) * 100,2)
            players[row['full_name']] = (score_player(row), cmp_pct)
            
            
    return pd.DataFrame([[key,value[0], value[1]] for key,value in players.iteritems()],columns=["Name","avg_fpg", "cmp_pct"]).sort_values('avg_fpg', ascending=False)

In [46]:
years = ['2017', '2018']
pos = ['QB']
df = get_qb_stats(years, pos)
df

,Name,avg_fpg,cmp_pct
16,Deshaun Watson,24.1,61.76
13,Russell Wilson,21.7,61.23
0,Carson Wentz,21.2,60.23
7,Alex Smith,19.7,67.52
1,Cam Newton,18.6,59.15
34,Aaron Rodgers,18.5,64.71
24,Tom Brady,18.4,66.21
12,Ben Roethlisberger,17.4,64.23
10,Kirk Cousins,17.4,64.26
38,Matthew Stafford,17.1,65.66


#### Gathering Team Level Statistics

Now that we have the relevant player stats, lets gather team level statistics.

In [67]:
# Get overall team offense rank

def get_team_stats(year_list):
    year_string = convert_list_sql_string(year_list)
    
    teams = {}

    offense_query = '''
    SELECT
          player.team as team,
          SUM(play_player.passing_yds) AS passing_yds,
          SUM(play_player.passing_tds) AS passing_tds,
          SUM(play_player.passing_tds) AS passing_cmp,
          SUM(play_player.rushing_yds) AS rushing_yds,
          SUM(play_player.rushing_tds) AS rushing_tds,
          SUM(play_player.receiving_tds) AS receiving_tds,
          SUM(play_player.receiving_yds) AS receiving_yds,
          SUM(play_player.receiving_rec) AS receiving_rec,
          COUNT(DISTINCT game.gsis_id) AS games_played
        FROM play_player
        LEFT JOIN game ON play_player.gsis_id = game.gsis_id
        LEFT JOIN player ON play_player.player_id = player.player_id
        WHERE game.season_type = 'Regular' AND game.season_year in (%s) AND player.team != 'UNK' 
        GROUP BY player.team
        
    ''' % year_string

    with nfldb.Tx(db) as cursor:
        cursor.execute(offense_query)
        for row in cursor.fetchall():
            teams[row['team']] = (score_player(row), row['passing_yds'])
    team_df = pd.DataFrame([[key,value[0], value[1]] for key,value in teams.iteritems()],columns=["Team","total_offense_score", "total_passing_yds"])
    team_df['total_offense_rank'] = team_df['total_offense_score'].rank(ascending=False).astype(int)
    team_df['pass_yds_rank'] = team_df['total_passing_yds'].rank(ascending=False).astype(int)
    return team_df

In [68]:
years = ['2017', '2018']
df = get_team_stats(years)
df

,Team,total_offense_score,total_passing_yds,total_offense_rank,pass_yds_rank
0,MIN,16.2,6378,7,1
1,MIA,9.0,1897,26,28
2,CAR,11.4,3312,20,22
3,ATL,19.4,4146,3,11
4,DET,11.3,4608,21,5
5,CIN,25.6,3320,2,21
6,NYJ,8.4,2957,29,26
7,DEN,11.1,4064,22,12
8,BAL,18.5,3179,5,23
9,NYG,7.1,3468,30,19
